```markdown
<img src="docs/images/DSPy8.png" alt="DSPy7 图片" height="150"/>

# DSPy: 在 SkyCamp 的教程
```

这个笔记本包含了 **SkyCamp 2023** 的 **DSPy 教程**。

让我们从设置开始。下面的代码片段将会在需要时安装 **DSPy**。

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # 当在谷歌Colab上时，让我们克隆笔记本以便下载缓存。
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# 为此笔记本设置缓存
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # 如果尚未安装该软件包，则安装该软件包
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai==2.1
    # !pip install -e $repo_path

!pip install transformers

In [ ]:
# 导入dspy库
import dspy
# 从dspy.evaluate模块中导入Evaluate类
from dspy.evaluate import Evaluate
# 从dspy.teleprompt模块中导入BootstrapFewShot、BootstrapFewShotWithRandomSearch、BootstrapFinetune类
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

### 1) 配置默认的语言模型和检索模型

我们将首先设置语言模型（LM）和检索模型（RM）。**DSPy**支持多种API和本地模型。

在这个笔记本中，我们将使用`Llama2-13b-chat`，使用HuggingFace TGI服务软件基础设施。原则上，您可以在自己的本地GPU上运行此模型，但是在本教程中，所有示例都是预缓存的，因此您不需要担心成本问题。

我们将使用检索模型`ColBERTv2`。为了简化操作，我们已经设置了一个ColBERTv2服务器，托管了一个维基百科2017年“摘要”搜索索引（即包含来自此[2017转储](https://hotpotqa.github.io/wiki-readme.html)的每篇文章的第一段），因此您不需要担心设置一个！而且是免费的。

**注意：** _如果按照说明运行此笔记本，则不需要API密钥。所有示例已经在内部缓存，因此您可以检查它们！_

In [ ]:
# 创建 HFClientTGI 对象 llama，使用模型"meta-llama/Llama-2-13b-chat-hf"，端口号为[7140, 7141, 7142, 7143]，最大 token 数为150
llama = dspy.HFClientTGI(model="meta-llama/Llama-2-13b-chat-hf", port=[7140, 7141, 7142, 7143], max_tokens=150)

# 创建 ColBERTv2 对象 colbertv2，指定 URL 为'http://20.102.90.50:2017/wiki17_abstracts'
colbertv2 = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

# # 注意: 在完成本笔记本后，如果你愿意，可以像这样使用 GPT-3.5。
# # 在这种情况下，如果你选择这样做，请确保在下面配置 lm=turbo。
# turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct')

# 配置 settings，rm 为 colbertv2，lm 为 llama
dspy.settings.configure(rm=colbertv2, lm=llama)

### 2) 为我们的任务创建一些问题-答案对

In [ ]:
# 定义一个包含问题和答案的训练集
train = [('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', 'Kevin Greutert'),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', 'Foxcatcher'),
         ('In what year was the star of To Hell and Back born?', '1925'),
         ('Which award did the first book of Gary Zukav receive?', 'U.S. National Book Award'),
         ('What documentary about the Gilgo Beach Killer debuted on A&E?', 'The Killing Season'),
         ('Which author is English: John Braine or Studs Terkel?', 'John Braine'),
         ('Who produced the album that included a re-recording of "Lithium"?', 'Butch Vig')]

# 使用dspy.Example类创建训练实例，每个实例包含问题和答案，并将问题作为输入
train = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in train]

In [ ]:
# 定义一个包含问题和答案的列表
dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', 'E. L. Doctorow'),
       ('Right Back At It Again contains lyrics co-written by the singer born in what city?', 'Gainesville, Florida'),
       ('What year was the party of the winner of the 1971 San Francisco mayoral election founded?', '1828'),
       ('Anthony Dirrell is the brother of which super middleweight title holder?', 'Andre Dirrell'),
       ('The sports nutrition business established by Oliver Cookson is based in which county in the UK?', 'Cheshire'),
       ('Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.', 'February 13, 1980'),
       ('Kyle Moran was born in the town on what river?', 'Castletown River'),
       ("The actress who played the niece in the Priest film was born in what city, country?", 'Surrey, England'),
       ('Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.', 'Portrait of a Marriage'),
       ('What year was the father of the Princes in the Tower born?', '1442'),
       ('What river is near the Crichton Collegiate Church?', 'the River Tyne'),
       ('Who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000?', 'Renault'),
       ('André Zucca was a French photographer who worked with a German propaganda magazine published by what Nazi organization?', 'the Wehrmacht')]

# 将问题和答案转换为Example对象，并指定输入为'question'
dev = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in dev]

### 3) 关键概念：签名和模块

In [ ]:
# 定义一个 dspy.Predict 模块，其签名为 `question -> answer`（即，接受一个问题并输出一个答案）。
predict = dspy.Predict('question -> answer')

# 使用该模块！
predict(question="What is the capital of Germany?")

在上面的示例中，我们使用了`dspy.Predict`模块的**零样本**，即在任何示例上都没有编译它。

现在让我们构建一个稍微更高级的程序。我们的程序将使用`dspy.ChainOfThought`模块，该模块要求LM逐步思考。

我们将称此程序为`CoT`。

In [ ]:
class CoT(dspy.Module):  # 定义一个新模块
    def __init__(self):
        super().__init__()

        # 声明思维链子模块，以便后续编译（例如，教它一个提示）
        self.generate_answer = dspy.ChainOfThought('question -> answer')
    
    def forward(self, question):
        return self.generate_answer(question=question)  # 在这里使用模块

现在让我们使用我们的七个“train”示例来编译这个。我们将在DSPy中使用非常简单的“BootstrapFewShot”。

In [ ]:
# 定义评估指标为精确匹配
metric_EM = dspy.evaluate.answer_exact_match

# 创建BootstrapFewShot对象，设置评估指标为精确匹配，最大bootstrap演示次数为2
teleprompter = BootstrapFewShot(metric=metric_EM, max_bootstrapped_demos=2)

# 编译CoT模型，使用训练集train进行训练
cot_compiled = teleprompter.compile(CoT(), trainset=train)

让我们向这个新程序提一个问题。

In [ ]:
# 调用cot_compiled函数，传入参数"What is the capital of Germany?"
cot_compiled("What is the capital of Germany?")

你可能会好奇发生了什么。让我们检查一下最后一次调用我们的 Llama 语言模型，看看提示和输出。

In [ ]:
# 使用llama.inspect_history(n=1)函数检查最近的1个历史记录
llama.inspect_history(n=1)

注意到提示以我们提出的问题结尾（"德国的首都是什么？"），但在此之前包含了少样本示例。

提示中的最后一个示例包含了一个自动生成的基于LM的理由（逐步推理），用作演示，针对训练问题"哪位作者是英国人：约翰·布雷恩还是斯塔兹·特克尔？"。

现在，让我们在开发集上进行评估。

In [ ]:
NUM_THREADS = 32  # 定义线程数为32
evaluate_hotpot = Evaluate(devset=dev, metric=metric_EM, num_threads=NUM_THREADS, display_progress=True, display_table=15)  # 创建评估对象evaluate_hotpot，传入开发集dev、评估指标metric_EM、线程数NUM_THREADS、显示进度display_progress为True、显示表格行数display_table为15

首先，让我们使用 Llama 评估编译后的 `CoT` 程序。请随意将下面的 `cot_compiled` 替换为 `CoT()`（注意括号），以测试 CoT 的零-shot版本。

In [ ]:
# 评估热锅数据
evaluate_hotpot(cot_compiled)

### 4) 奖励1：带有查询生成的RAG

作为奖励，让我们定义一个更复杂的程序称为`RAG`。这个程序将会：

- 使用语言模型（LM）基于输入问题生成一个搜索查询
- 使用我们的检索器检索三个段落
- 使用语言模型（LM）使用这些段落生成最终答案

In [ ]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        # 声明三个模块：检索器、查询生成器和答案生成器
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_query = dspy.ChainOfThought("question -> search_query")
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        # 从问题生成搜索查询，并用它检索段落
        search_query = self.generate_query(question=question).search_query
        passages = self.retrieve(search_query).passages

        # 从段落和问题生成答案
        return self.generate_answer(context=passages, question=question)

出于好奇，我们可以评估这个程序的**未编译**（或**零射击**）版本。

In [ ]:
# 调用evaluate_hotpot函数，传入RAG()作为参数，并设置display_table参数为0
evaluate_hotpot(RAG(), display_table=0)

让我们现在编译这个RAG程序。这次我们将使用一个稍微更高级的话筒提示器（自动提示优化器），它依赖于随机搜索。

In [ ]:
# 创建一个BootstrapFewShotWithRandomSearch对象，指定metric为metric_EM，最大bootstrapped示例数为2，候选程序数为8，线程数为NUM_THREADS
teleprompter2 = BootstrapFewShotWithRandomSearch(metric=metric_EM, max_bootstrapped_demos=2, num_candidate_programs=8, num_threads=NUM_THREADS)

# 使用teleprompter2编译RAG模型，指定训练集为train，验证集为dev
rag_compiled = teleprompter2.compile(RAG(), trainset=train, valset=dev)

让我们现在评估这个编译版本的RAG。

In [ ]:
# 调用 evaluate_hotpot 函数并传入 rag_compiled 参数
evaluate_hotpot(rag_compiled)

让我们检查其中一个语言模型调用。特别关注提示中最后几个输入/输出示例的结构。

In [ ]:
# 调用rag_compiled函数，提出问题："1971年旧金山市长选举的获胜者的政党是在哪一年成立的？"
rag_compiled("What year was the party of the winner of the 1971 San Francisco mayoral election founded?")

# 调用llama对象的inspect_history方法，查看最近1次的历史记录
llama.inspect_history(n=1)

### 4) 奖励2：多跳检索和推理

让我们现在构建一个简单的多跳程序，该程序将交替调用语言模型和检索器。

请按照下面的**TODO**说明来实现这个。

In [ ]:
from dsp.utils.utils import deduplicate

class MultiHop(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_query = dspy.ChainOfThought("question -> search_query")

        # TODO: 定义一个带有签名 'context, question -> search_query' 的 dspy.ChainOfThought 模块。
        self.generate_query_from_context = None

        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        passages = []
        
        search_query = self.generate_query(question=question).search_query
        passages += self.retrieve(search_query).passages

        # TODO: 用调用 self.generate_query_from_context 替换 `None`，生成一个搜索查询。
        # 注意：在 DSPy 中，始终向模块传递关键字参数（例如，context=...，question=...）以避免歧义。
        # 注意2：不要忘记访问字段 .search_query 以从模块的输出中提取它。
        # 注意3：查看以下笔记本以查看一个完成的示例：https://github.com/stanfordnlp/dspy/blob/main/skycamp2023_completed.ipynb。
        search_query2 = None

        # TODO: 用调用 self.retrieve 替换 `None`，检索段落。将它们附加到列表 `passages` 中。
        passages += None

        return self.generate_answer(context=deduplicate(passages), question=question)

In [ ]:
# 编译 MultiHop 模型并使用 train 数据集进行训练，使用 dev 数据集进行验证
multihop_compiled = teleprompter2.compile(MultiHop(), trainset=train, valset=dev)

In [ ]:
# 调用 evaluate_hotpot 函数，传入 multihop_compiled 参数和 devset 参数
evaluate_hotpot(multihop_compiled, devset=dev)

让我们现在检查一个问题的第二跳搜索查询的提示。

In [ ]:
# 调用multihop_compiled函数，传入问题参数
multihop_compiled(question="Who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000?")
# 调用llama对象的inspect_history方法，设置参数n=1, skip=2
llama.inspect_history(n=1, skip=2)